# Design Primers from Aquarium

## Imports

In [2]:
import sys
import os

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

['/home/justin/Github/aqbuildtools',
 '/home/justin/Github/aqbuildtools/examples/2021RedesignCampaign',
 '/home/justin/anaconda3/envs/aqbt/lib/python38.zip',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/lib-dynload',
 '',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages/IPython/extensions',
 '/home/justin/.ipython']

# Utils

In [ ]:
import pydent

def create_sample_from_row(row):
    stype = list(row)[0]
    sample_type = aq.SampleType.find_by_name(stype)
    name = row[sample_type.name]
    description = row.get('Description', '')
    project = row['Project']
    property_rows = {k: v for k, v in row.items() if k not in [stype, 'Project', 'Description'] and not k.startswith('_')}
    
    ftdict = {ft.name: ft for ft in sample_type.field_types}
    properties = {}
    for k, v in property_rows.items():
        if ftdict[k].ftype == 'sample':
            if isinstance(v, int):
                v = aq.Sample.find(v)
                assert v
            elif isinstance(v, str):
                v = aq.Sample.find_by_name(v)
                assert v
            elif isinstance(v, pydent.models.Sample):
                pass
            else:
                raise TypeError
        properties[k] = v
    return aq.Sample.new(sample_type_id=sample_type.id, name=name, description=description, project=project, properties=properties)
        
def create_row_from_sample(sample):
    sample_type = sample.sample_type
    ftdict = {ft.name: ft for ft in sample_type.field_types}
    row = {
        sample_type.name: sample.name,
        'Description': sample.description,
        'Project': sample.project,
    }
    props = sample.properties
    for k, v in ftdict.items():
        if ftdict[k].ftype == 'sample':
            if k in props and props[k]:
                v = props[k].name
            else:
                v = ''
        else:
            v = props[k]
        row[k] = v
    return row

# Designs

### Load sessions

In [23]:
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

### Design LEU 3' Homology

Design the 3' homology from the genome

In [21]:
from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils

url = 'https://benchling.com/s/seq-kJsKxNoX8ecE8b198wcD'
benchling = aqtools.sessions['production']['benchling']
template = 'aaagtaagaaaagatcattagaagataatgaaactgaaattgaggtatcccgagacacatggaataataagaatatgagaagtctggaaccaccaagatcgaagaaacgcataaatttaattgcagcaataaaaggagtgaaatcgatcaaaccagttcgaacgaccttaagatatgatgaagcaattacttataataaagacaacaaagaaaaagacagatatgttgaagcttatcataaagaaattagccaactattgaaaatgaacacttgggatacaaacaaatattatgatagaaatgacatagatcctaaaaaagtaataaactcaatgtttatatttaacaagaaacgtgatggtacacacaaagctagatttgttgcaagaggcgacattcaacaccccgatacatatgattctgatatgcaatccaataccgtacatcactatgcactgatgacgtcactgtcaatcgcattagacaacgactattatatcacacagctggacatatcctctgcttacttatatgctgatatcaaagaagaattatacataagacctccaccacatttaggtttgaatgataaattactacgtttgagaaaatcactctatggtttgaaacaaagtggtgcaaactggtatgaaaccattaaatcatatttaataaattgttgcgacatgcaagaagttcgcggatggtcatgcgtatttaagaatagtcaagtaacaatttgcttattcgttgatgatatgatattattcagcaaagacttaaatgcaaataagaaaatcataacaacactcaagaaacaatacgatacaaagataataaatctgggtgaaagtgataacgaaattcagtacgacatacttggattagagatcaaatatcaaagaagcaagtacatgaaattaggtatggaaaaatccttgacagaaaaattacccaaactaaacgtacctttgaacccaaaaggaaagaaacttagagctccaggtcaaccaggtcattatatagaccaggatgaactagaaatagatgaagatgaatacaaagagaaagtacatgaaatgcaaaagttgattggtctagcttcatatgttggatataaatttagatttgacttactatactacatcaacacacttgctcaacatatactattcccctctaggcaagttttagacatgacatatgagttaatacaattcatgtgggacactagagataaacaattaatatggcacaaaaacaaacctaccaagccagataataaactagtcgcaataagcgatgcttcatatggtaaccaaccatattacaagtcacaaattggtaacattttcctactcaacggaaaagtgattggaggaaagtcgacaaaggcttcgttaacatgcacttcaactacagaagcagaaatacacgcagtcagtgaagctattccgctattgaataacctcagtcaccttgtgcaagaacttaacaagaaaccaattattaaaggcttacttactgatagtagatcaacgatcagtataattaagtctacaaatgaagagaaatttagaaacagattttttggcacaaaggcaatgagacttagagatgaagtatcaggtaataatttatacgtatactacatcgagaccaagaagaacattgctgatgtgatgacaaaacctcttccgataaaaacatttaaactattaactaacaaatggattcattagatctattacattatgggtggtatgttggaataaaaatcaactatcatctactaactagtatttacgttactagtatattatcatatacggtgttagaagatgacgcaaatgatgagaaatagtcatctaaattagtggaagctgaaacgcaaggattgataatgtaataggatcaatgaatattaacatataaaatgatgataataatatttatagaattgtgtagaattgcagattcccttttatggattcctaaat'

import primer3plus

design = primer3plus.new()

design.settings.as_generic_task()
design.settings.template('aaagtaagaaaagatcattagaagataatgaaactgaaattgaggtatcccgagacacatggaataataagaatatgagaagtctggaaccaccaagatcgaagaaacgcataaatttaattgcagcaataaaaggagtgaaatcgatcaaaccagttcgaacgaccttaagatatgatgaagcaattacttataataaagacaacaaagaaaaagacagatatgttgaagcttatcataaagaaattagccaactattgaaaatgaacacttgggatacaaacaaatattatgatagaaatgacatagatcctaaaaaagtaataaactcaatgtttatatttaacaagaaacgtgatggtacacacaaagctagatttgttgcaagaggcgacattcaacaccccgatacatatgattctgatatgcaatccaataccgtacatcactatgcactgatgacgtcactgtcaatcgcattagacaacgactattatatcacacagctggacatatcctctgcttacttatatgctgatatcaaagaagaattatacataagacctccaccacatttaggtttgaatgataaattactacgtttgagaaaatcactctatggtttgaaacaaagtggtgcaaactggtatgaaaccattaaatcatatttaataaattgttgcgacatgcaagaagttcgcggatggtcatgcgtatttaagaatagtcaagtaacaatttgcttattcgttgatgatatgatattattcagcaaagacttaaatgcaaataagaaaatcataacaacactcaagaaacaatacgatacaaagataataaatctgggtgaaagtgataacgaaattcagtacgacatacttggattagagatcaaatatcaaagaagcaagtacatgaaattaggtatggaaaaatccttgacagaaaaattacccaaactaaacgtacctttgaacccaaaaggaaagaaacttagagctccaggtcaaccaggtcattatatagaccaggatgaactagaaatagatgaagatgaatacaaagagaaagtacatgaaatgcaaaagttgattggtctagcttcatatgttggatataaatttagatttgacttactatactacatcaacacacttgctcaacatatactattcccctctaggcaagttttagacatgacatatgagttaatacaattcatgtgggacactagagataaacaattaatatggcacaaaaacaaacctaccaagccagataataaactagtcgcaataagcgatgcttcatatggtaaccaaccatattacaagtcacaaattggtaacattttcctactcaacggaaaagtgattggaggaaagtcgacaaaggcttcgttaacatgcacttcaactacagaagcagaaatacacgcagtcagtgaagctattccgctattgaataacctcagtcaccttgtgcaagaacttaacaagaaaccaattattaaaggcttacttactgatagtagatcaacgatcagtataattaagtctacaaatgaagagaaatttagaaacagattttttggcacaaaggcaatgagacttagagatgaagtatcaggtaataatttatacgtatactacatcgagaccaagaagaacattgctgatgtgatgacaaaacctcttccgataaaaacatttaaactattaactaacaaatggattcattagatctattacattatgggtggtatgttggaataaaaatcaactatcatctactaactagtatttacgttactagtatattatcatatacggtgttagaagatgacgcaaatgatgagaaatagtcatctaaattagtggaagctgaaacgcaaggattgataatgtaataggatcaatgaatattaacatataaaatgatgataataatatttatagaattgtgtagaattgcagattcccttttatggattcctaaat')
design.settings.included((0, len(template)))
design.settings.product_size((300, 700))
design.PRIMER_PICK_ANYWAY = False
design.PRIMER_MIN_ANNEAL_CHECK = 14
design.SEQUENCE_FORCE_RIGHT_START = len(template) - 1
design.settings.use_overhangs()
design.settings.long_ok()
design.run_and_optimize(10)

({0: {'PAIR': {'PENALTY': 9.918311215714084,
    'COMPL_ANY_TH': 0.0,
    'COMPL_END_TH': 0.0,
    'PRODUCT_SIZE': 533},
   'LEFT': {'PENALTY': 0.5461219135515876,
    'SEQUENCE': 'acctcagtcaccttgtgcaa',
    'location': (1467, 20),
    'TM': 59.45387808644841,
    'GC_PERCENT': 50.0,
    'SELF_ANY_TH': 0.0,
    'SELF_END_TH': 0.0,
    'HAIRPIN_TH': 32.981122668144224,
    'END_STABILITY': 4.08,
    'OVERHANG': ''},
   'RIGHT': {'PENALTY': 9.372189302162496,
    'SEQUENCE': 'atttaggaatccataaaagggaatctg',
    'location': (1999, 27),
    'TM': 57.6278106978375,
    'GC_PERCENT': 33.333333333333336,
    'SELF_ANY_TH': 0.6076452362053146,
    'SELF_END_TH': 0.0,
    'HAIRPIN_TH': 38.09337512031675,
    'END_STABILITY': 2.9,
    'OVERHANG': ''}},
  1: {'PAIR': {'PENALTY': 9.918311215714084,
    'COMPL_ANY_TH': 0.0,
    'COMPL_END_TH': 0.0,
    'PRODUCT_SIZE': 534},
   'LEFT': {'PENALTY': 0.5461219135515876,
    'SEQUENCE': 'aacctcagtcaccttgtgca',
    'location': (1466, 20),
    'TM': 59.4538

### Standard Homology Designs

In [29]:
from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils

primer_designer = primer_utils.AqPrimerDesign(aq)

def create_primer_row(name, data):
    primer_data = {
        'Primer': name,
        'Project': "SD2 2021 Designs",
        'Overhang Sequence': data['OVERHANG'],
        'Anneal Sequence': data['SEQUENCE'],
        'T Anneal': round(data['TM'], 2)
    }
    return primer_data

def rows_to_df(rows):
    df = pd.DataFrame(rows)
    for c in df.columns:
        if c.startswith('_'):
            df = df.drop(c, axis=1)
    return df

class Templates():
    trp_3_hom = 'cgctcgaaggctttaatttgatgtcgtaataaccccgccccgtgcaggccttttgaaaagcaagcataaaagatctaaacataaaatctgtaaaataacaagatgtaaagataatgctaaatcatttggctttttgattgattgtacaggaaaatatacatcgcagggggttgacttttaccatttcaccgcaatggaatcaaacttgttgaagagaatgttcacaggcgcatacgctacaatgacccgattcttgctagccttttctcggtcttgcaaacaaccgccggcagcttagtatataaatacacatgtacatacctctctccgtatcctcgtaatcattttcttgtatttatcgtcttttcgctgtaaaaactttatcacacttatctcaaatacacttattaaccgcttttactattatcttctacgctgacagtaatatcaaacagtgacacatattaaacacagtggtttctttgcataaacaccat'
    ura_3_hom = 'cgctcgaaggctttaatttgatgtcgtaataaccccgccccgaaaactgtattataagtaaatgcatgtatactaaactcacaaattagagcttcaatttaattatatcagttattacccgggaatctcggtcgtaatgatttctataatgacgaaaaaaaaaaaattggaaagaaaaagcttcatggcctttataaaaaggaactatccaatacctcgccagaaccaagtaacagtattttacggggcacaaatcaagaacaataagacaggactgtaaagatggacgcattgaactccaaagaacaacaagagttccaaaaagtagtggaacaaaagcaaatgaaggatttcatgcgtttgtactctaatctggtagaaagatgtttcacagactgtgtcaatgacttcacaacatcaaagctaaccaataaggaacaaacatgcatcatgaagtgctcagaaaagttcttgaagcatagcgaacgtgtagggcagcgtttccaagaacaaaacgctgccttgggacaaggcttgggccg'
    his_3_hom = 'cgctcgaaggctttaatttgatgtcgtaataaccccgccccgtgacaccgattatttaaagctgcagcatacgatatatatacatgtgtatatatgtatacctatgaatgtcagtaagtatgtatacgaacagtatgatactgaagatgacaaggtaatgcatcattctatacgtgtcattctgaacgaggcgcgctttccttttttctttttgctttttctttttttttctcttgaactcgagaaaaaaaatataaaagagatggaggaacgggaaaaagttagttgtggtgataggtggcaagtggtattccgtaagaacaacaagaaaagcatttcatattatggctgaactgagcgaacaagtgcaaaatttaagcatcaacgacaacaacgagaatggttatgttcctcctcacttaagaggaaaaccaagaagtgccagaaataacagtagcaactacaataacaacaacggcggctacaacggtggccgtggcggtggcagcttctttagcaacaaccgtcgtggtggttacggcaacggtggtttcttcggtggaaacaacggtggcagcagatctaacggccgttctggtggtagatggatcgatggcaaacatgtcccagctccaagaaacgaaaaggccgagatcgccatatttggtgtccccgaggatccaaatttccaatcttctggtattaacttcgataactacgatgatattccagtggacgcctctggtaaggatgttcctgaaccaatcacagaatttacctcacctccattggacggattgttattggaaaacatcaaattggcccgtttcaccaagccaacacctgtgcaaaaatactccgtccctatcgttgccaacggcagagatttgatggcctgtgcgcagaccggttctggtaagactggtgggtttttattcccagtgttgtccgaatcatttaagactggaccatctcctcaaccagagtctcaaggctccttttaccaaagaaaggcctacccaactgctgtcattatggctcca'
    HO_3_hom = 'acgccattttaagtccaaaggcacaattttacgttggaactaaacggtaaagatcaaatatcacctttattcttttttggtatgcatcgatttctttgccagtaagaactacgtgcaactctacattagatcaaaatataacggatcataaatgcaaacaacatttgtccaaaagaaacaaaaatgttggcgtttgtctcgaacatgacacaatactccttgatcgagggacattgcccgattaactatgatttatactttttactacaggtactcgttctccctactatttcaccagtatacggtcgttttatgattcatttattgcctataatggtaaaaagggacatcttaaccttacgtgttgcaatgagaccattcttttcgattcatcaactgtgaaatttctgaaatccagtacaatgcgaaacgctaccgataatggcaccgtcttttgaattatgaggcccagatatggcgttcaatcttcaacaaaaaaatgaatatcaaaactaaattctatctggataacaatctttctttggtttgcagaaatatcattaaaaaacaatccataactactaaggaagggtataaatggtttggtttactaattttatattttgatttgattagcaattataatctctaatatactatttctgtagccaaaaatctgcaatgtcaataagtagcaagtgccgcttactcgtgagaatatcaaccttatagcatatgttttctattttttttttttgcttatggagataatgaacattgtacacatgaacaaagtggtagttctctctccttgattttttgtgcttgcgatatatataattgttacattcaatggccatgctaaataaaggttttagctttgtttcactcgtgcccaggataatt'
    HO_5_hom = 'TTAGCAGATGCGCGCACCTGCGTTGTTACCACAACTCTTATGAGGCCCGCGGACAGCATCAAActgtaagattccgccacattttatacactctggtcctttaactggcaaaccttcgggcgtaatgcccaatttttcgcctttgtcttttgcctttttcacttcacgtgcttctggtacatacttgcaatttatacagtgatgaccgctgaatttgtatcttccatagcatctagcacatactcgatttttaccactccaatctttataaaaatacttgattccctttctgggacaagcaacacagtgttttagattctttttttgtgatattttaagctgttctcccacacagcagcctcgacatgatttcacttctattttgttgccaagcaagaaatttttatggccttctatcgtaagcccatatacagtactctcaccctggaaatcatccgtgaagctgaaatatacgggttccctttttataattggcggaacttctcttgttttgtgaccacttcgacaatatgacaaaacattctgtgaagttgttcccccagcatcagagcagattgtactgagagtgcaccGGCGCGCCAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGGGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTAAGGAAAAGACTCACGTTTCGAGGCCGCGATTAAATTCCAACATGGATGCTGATTTATATGGGTATAAATGGGCTCGCGATAATGTCGGGCAATCAGGTGCGACAATCTATCGATTGTATGGGAAGCCCGATGCGCCAGAGTTGTTTCTGAAACATGGCAAAGGTAGCGTTGCCAATGATGTTACAGATGAGATGGTCAGACTAAACTGGCTGACGGAATTTATGCCTCTTCCGACCATCAAGCATTTTATCCGTACTCCTGATGATGCATGGTTACTCACCACTGCGATCCCCGGCAAAACAGCATTCCAGATATTAGAAGAATATCCTGATTCAGGTGAAAATATTGTTGATGCGCTGGCAGTGTTCCTGCGCCGGTTGCATTCGATTCCTGTTTGTAATTGTCCTTTTAACAGCGATCGCGTATTTCGTCTCGCTCAGGCGCAATCACGAATGAATAACGGTTTGGTTGATGCGAGTGATTTTGATGACGAGCGTAATGGCTGGCCTGTTGAACAAGTCTGGAAAGAAATGCATAAGCTTTTGCCATTCTCACCGGATTCAGTCGTCACTCATGGTGATTTCTCACTTGATAACCTTATTTTTGACGAGGGGAAATTAATAGGTTGTATTGATGTTGGACGAGTCGGAATCGCAGACCGATACCAGGATCTTGCCATCCTATGGAACTGCCTCGGTGAGTTTTCTCCTTCATTACAGAAACGGCTTTTTCAAAAATATGGTATTGATAATCCTGATATGAATAAATTGCAGTTTCATTTGATGCTCGATGAGTTTTTCTAATCAGTACTGACAATAAAAAGATTCTTGTTTTCAAGAACTTGTCATTTGTATAGTTTTTTTATATTGTAGTTGTTCTATTTTAATCAAATGTTAGCGTGATTTATATTTTTTTTCGCCTCGACATCATCTGCCCAGATGCGAAGTTAAGTGCGCAGAAAGTAATATCATGCGTCAATCGTATGTGAATGCTGGTCGCTATACTGCTGTCGATTCGATACTAACGCCGCCATCCAGTGTC'
    ARS314_3_hom = 'CCTGAGGGGCGCGCCAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCatgggtatgaccgaccaagcgacgcccaacctgccatcacgagatttcgatcccaccgccgccttctatgaaaggttgggcttcggaatcgttttccgggacgccggctggatgatcctccagcgcggggatctcatgctggagttcttcgcccaccccgggctcgatcccctcgcgagttggttcagctgctgcctgaggctggacgacctcgcggagttctaccggcagtgcaaatccgtcggcatccaggaaaccagcagcggctatccgcgcatccatgcccccgaactgcaggagtggggaggcacgatggccgctttggtcgacccggacgggacgctcctgcgcctgatacagaacgaattgcttgcaggcatctcatgaTCAGTACTGACAATAAAAAGATTCTTGTTTTCAAGAACTTGTCATTTGTATAGTTTTTTTATATTGTAGTTGTTCTATTTTAATCAAATGTTAGCGTGATTTATATTTTTTTTCGCCTCGACATCATCTGCCCAGATGCGAAGTTAAGTGCGCAGAAAGTAATATCATGCGTCAATCGTATGTGAATGCTGGTCGCTATACTGCTGTCGATTCGATACTAACGCCGCCATCCAGTGTCCAGAAGCGAGGCGAATAAAGGTGGCATTATCTACTGTAGTTATGCAAGTATTCTTAATTTTCAGTACTAAACATGAATATTAATTATTTGTGAGCGTACAAGAAGCAAAAGGAAGTCATCTTTCTTTGAACAGTTCTGTTTTTAGGTAAGCCTTGGAACCTTAGCTATACTGACTTGATACGCCCTGTCTTTGTTGTCGTTATATAAAGTGTGCACCTTTATCATAGCCCCTAATAAGCTTCTTGCAGTAGTATTTGAACAGTGTATATCTTCATATTGCTTTTGTTGTAAATATTGCGCAAAAGCGACTATCTTTGTGATTTGGTTGAACTTTTGCATATTCACGTTATTACCATCTTTAGTGAATGTATCGTTTCCGTCTCTTATGAAGGTAATGTCCCTGATAAGCAGTGATGTAAAAGGTACACAAGGAAGCTGACTGTGGAAAATATGTTTAATTGTTCTTCTATAAACGTTGTAGTTGTTGTTTGGATGTACGACGACTTTTAGGCGCTGAAAAAGGTGGTCTGATTTAACATCTATCGGGAGAGATAGTCTTTCTATTGAATGATTTTGCAATGATGTAATGATTGAAGCAAGTGAGTTTAAGTTTCGTAAGTATAGACAAGACAGTGCAACTTGTAACCAGTAGGAAATGGTGTGCGTTTGCTGCAAAGTTGTCTCAAGAACGTAGGACGATAACTGGTTGGAAAGCGTAAACACGGAGTCAATTGTTGTATCGTTGTGTTTGAAATGTCTTGTGAATTCTATTGTTTCAATATCCAAGTAAAGAGAAGATTCTAAGAGAGTCAATGTCTTTGCAAGTGACCATGGAGATACGTTTAGAGCGAAGGCTTCGACTTCTAGAAGTTCACCCTGTTTCCATTGGAAAAGTAAATCACTGAGGTCAGTTGCACCGCACAATTCATCATTTGCGTTCGTTTTGTCTATCTCGTATTGGAGAAATAAGTTTTCGTAGTCCAGGGGCGGTTTATTTTCTTCGA'
    ARS314_5_hom = 'TGAATGTGTCTCTGATGTACCAGAAATAATTTCTCTTAAAGGAGGCAATGCTTGAATTTCAGGATTGAAAGTTTCATTCAATCTTGTTACAATGCCTTCCTTCTCCAAATCATTAATCAAGTGGTCAAATCTTTCATAGAACTTAGCTTCCATTCTTTTGTAGAAATCATCTACTTTCAATTTCTCGTCGCTCAGGTTAGTTACGAATGTATCATAAGAACTGAAGAGCGTCTTTCCCTCGCTACTCACAGATGAGCTGCGTGATTCAAAGAATTTCTTACGTATAAAATTCTTATTGAAAGAGGTGGTTTTCCCTTGAGGAGATAACTCGTCTAACTCAATTGTTTCCTCGTTAATATTGTTCCCATCTATGGCCTTTTCGTCGGAGTCTCCCCTTTTGCTTCCTGAAGGTGTTTTAGAACCAAAGAGCTTACGCTTGAGTTTATGTGTAAATCTTCTTTTCGATGAAGATGGTTCTCCTGCAGCTATGTTGCTTTCTATATCAGCGATTGGACCTGGAGTCTGAGAGTCAGCGTCATCGTTTAAGTCACCGGTTGGCGTTTCTTGTTTCAATTCATCTGTCTGTAATGTGTAGATCAAATTTTTCAATTCGTTATAATCTAGGTAATGATTCTGCCATTCAGGGACAGCGTTGTATTTGAGAAAGTGTGAGAATCTCATTTCGAGGCACAATAATAGCCGCCTTTACCGTAGTTTTGCTGCACCTTTATCTGAGAGCTGACTGCTTTTTTGGTGTGAAACACTGTTTTCTGGTAATAATTTTTCAATGCATCGGATTACTTTTCCCACGTGCGAAATCATCAATTAATTAGATTGAAAAAAGGGTAAGGGAAAATAAGAAAGAGGCGGTAGCCTAAAGATACGGTAATTGAAACGTTTCCTATGCACAATCTTAAACCTTTTTAGGTAATTGATTAAGTTGACTGTAATATCTGTAAAAGATTACATCTAATTTACGTCTGGTTTCT'
    
    # Chr III 92895 - 90674
    LEU_5_hom = 'tcgactacgtcgtaaggccgtttctgacagagtaaaattcttgagggaactttcaccattatgggaaatggttcaagaaggtattgacttaaactccatcaaatggtcaggtcattgagtgttttttatttgttgtatttttttttttttagagaaaatcctccaatatataaattaggaatcatagtttcatgattttctgttacacctaactttttgtgtggtgccctcctccttgtcaatattaatgttaaagtgcaattctttttccttatcacgttgagccattagtatcaatttgcttacctgtattcctttacatcctcctttttctccttcttgataaatgtatgtagattgcgtatatagtttcgtctaccctatgaacatattccattttgtaatttcgtgtcgtttctattatgaatttcatttataaagtttatgtacaaatatcataaaaaaagagaatctttttaagcaaggattttcttaacttcttcggcgacagcatcaccgacttcggtggtactgttggaaccacctaaatcaccagttctgatacctgcatccaaaacctttttaactgcatcttcaatggccttaccttcttcaggcaagttcaatgacaatttcaacatcattgcagcagacaagatagtggcgatagggtcaaccttattctttggcaaatctggagcagaaccgtggcatggttcgtacaaaccaaatgcggtgttcttgtctggcaaagaggccaaggacgcagatggcaacaaacccaaggaacctgggataacggaggcttcatcggagatgatatcaccaaacatgttgctggtgattataataccatttaggtgggttgggttcttaactaggatcatggcggcagaatcaatcaattgatgttgaaccttcaatgtagggaattcgttcttgatggtttcctccacagtttttctccataatcttgaagaggccaaaacattagctttatccaaggaccaaataggcaatggtggctcatgttgtagggccatgaaagcggccattcttgtgattctttgcacttctggaacggtgtattgttcactatcccaagcgacaccatcaccatcgtcttcctttctcttaccaaagtaaatacctcccactaattctctgacaacaacgaagtcagtacctttagcaaattgtggcttgattggagataagtctaaaagagagtcggatgcaaagttacatggtcttaagttggcgtacaattgaagttctttacggatttttagtaaaccttgttcaggtctaacactaccggtaccccatttaggaccacccacagcacctaacaaaacggcatcaGccttcttggaggcttccagcgcctcatctggaagtggAacacctgtagcatcgatagcagcaccaccaattaaatgattttcgaaatcgaacttgacattggaacgaacatcagaaatagctttaagaaccttaatggcttcggctgtgatttcttgaccaacgtggtcacctggcaaaacgacgatcttcttaggggcagacattagaatggtatatccttgaaatatatatatatattgctgaaatgtaaaaggtaagaaaagttagaaagtaagacgattgctaaccacctattggaaaaaacaataggtccttaaataatattgtcaacttcaagtattgtgatgcaagcatttagtcatgaacgcttctctattctatatgaaaagccggttccggcGctctcacctttcctttttctcccaatttttcagttgaaaaaggtatatgcgtcaggcgacctctgaaattaacaaaaaatttccagtcatcgaatttgattctgtgcgatagcgcccctgtgtgttctcgttatgttgaggaaaaaaataatggttgctaagagattcgaactcttgcatcttacgatacctgagtattcccacagttaactgcggtcaagatatttcttgaatcaggcgccttagaccgctcggccaaacaaccaattacttgttgagaaatagagtataattatcctataaatataacgtttttgaacacacatgaacaaggaagtacaggacaattgattttgaagagaatgtggattttgatgtaattgttgggattccatttttaataaggcaataatattaggtatgtggatatactagaagttctcctcgagggtcgatatgc'

class Junctions():
    TS = 'atgtcgtaataaccccgccccg'
    PP2 = 'ccttaaccagattcgaaaagcggc'

#### Create Design Table

In [30]:
import pandas as pd


sample_default = {
    'Project': 'SD2 2021 Designs'
}

default_fragment_row = {
    'Sequence': '',
    'Restriction Enzyme(s)': '',
    'Yeast Marker': '',
    'Fragment Mix Array': '',
}


rows = [
    {
        "Fragment": "tCyc1(partial)-TS-TRP_3'hom__v3",
        'Template': 'pMOD TRP pGalZ4::NLS-eYFP-JAZ12',
        '_template_sequence': Templates.trp_3_hom,
        '_fwd_primer': aq.Sample.find_by_name('j5_0436_(pMODkan-HO)_forward'),
        'Reverse Primer': "3'_TRP_hom__rev"
    },
    {
        "Fragment": "tCyc1(partial)-TS-URA_3'hom__v3",
        "Template": 'pMOD6-pGRR-RGR-W8',
        '_template_sequence': Templates.ura_3_hom,
        '_fwd_primer': aq.Sample.find_by_name('j5_0436_(pMODkan-HO)_forward'),
        'Reverse Primer': "3'_URA_hom__rev"
    },
    {
        "Fragment": "tCyc1(partial)-TS-HIS_3'hom__v3",
        "Template": 'pMOD8A-URGR-W8',
        '_template_sequence': Templates.his_3_hom,
        '_fwd_primer': aq.Sample.find_by_name('j5_0436_(pMODkan-HO)_forward'),
        'Reverse Primer': "3'_HIS_hom__rev"
    },
    {
        "Fragment": "TS-HO_3'hom__v3",
        "Template": "pMOD-HO-KanMX-pRPL18B-rtTA-VPR",
        "_template_sequence": Templates.HO_3_hom,
        'Forward Primer': "TS-3'_HO_hom__fwd",
        'Reverse Primer': "3'_HO_hom__rev",
        "_lflank": Junctions.TS
    },
    {
        "Fragment": "HO-KAN-pp2_5'hom__v3",
        "Template": "pMOD-HO-KanMX-pRPL18B-rtTA-VPR",
        "_template_sequence": Templates.HO_5_hom,
        'Forward Primer': "5'_HO_hom__fwd",
        'Reverse Primer': "5'_HO_hom-pp2__rev",
        "_rflank": Junctions.PP2
    },
        {
        "Fragment": "TS-ARS314-BLEO_3'hom__v3",
        "Template": "pyMOD-BLEO-ARS314.A.0-pGPD-yeVenus-tCYC1",
        "_template_sequence": Templates.ARS314_3_hom,
        'Forward Primer': "TS-3'_ARS314_hom__fwd",
        'Reverse Primer': "3'_ARS314_hom__rev",
        "_lflank": Junctions.TS
    },
    {
        "Fragment": "ARS314-pp2_5'hom__v3",
        "Template": "pyMOD-BLEO-ARS314.A.0-pGPD-yeVenus-tCYC1",
        "_template_sequence": Templates.ARS314_5_hom,
        'Forward Primer': "5'_ARS314_hom__fwd",
        'Reverse Primer': "5'_ARS314_hom-pp2__rev",
        "_rflank": Junctions.PP2
    },
    {
        'Fragment': "LEU-pp2_5'hom__v3",
        "Template": 'pMODL-pGPD-AFB2',
        '_template_sequence': Templates.LEU_5_hom,
        'Forward Primer': "5'_LEU_hom__fwd",
        'Reverse Primer': "5'_LEU_hom-pp2__rev",
        '_rflank': Junctions.PP2
    }
]
rows_to_df(rows)

,Fragment,Template,Reverse Primer,Forward Primer
0,tCyc1(partial)-TS-TRP_3'hom__v3,pMOD TRP pGalZ4::NLS-eYFP-JAZ12,3'_TRP_hom__rev,NaN
1,tCyc1(partial)-TS-URA_3'hom__v3,pMOD6-pGRR-RGR-W8,3'_URA_hom__rev,NaN
2,tCyc1(partial)-TS-HIS_3'hom__v3,pMOD8A-URGR-W8,3'_HIS_hom__rev,NaN
3,TS-HO_3'hom__v3,pMOD-HO-KanMX-pRPL18B-rtTA-VPR,3'_HO_hom__rev,TS-3'_HO_hom__fwd
4,HO-KAN-pp2_5'hom__v3,pMOD-HO-KanMX-pRPL18B-rtTA-VPR,5'_HO_hom-pp2__rev,5'_HO_hom__fwd
5,TS-ARS314-BLEO_3'hom__v3,pyMOD-BLEO-ARS314.A.0-pGPD-yeVenus-tCYC1,3'_ARS314_hom__rev,TS-3'_ARS314_hom__fwd
6,ARS314-pp2_5'hom__v3,pyMOD-BLEO-ARS314.A.0-pGPD-yeVenus-tCYC1,5'_ARS314_hom-pp2__rev,5'_ARS314_hom__fwd
7,LEU-pp2_5'hom__v3,pMODL-pGPD-AFB2,5'_LEU_hom-pp2__rev,5'_LEU_hom__fwd


In [43]:
primer_rows = []

for row in rows:
    kwargs = {
        "template": row['_template_sequence'],
    }
    if '_fwd_primer' in row:
        # primer_designer.design_cloning_primers(template=Templates.trp_3_hom, fwd_primer='CGCTCGAAGGCTTTAATTTGATGTCGTAATAACCCCGCCCCG', fwd_primer_meta)
        fwd_primer = row['_fwd_primer']
        primer_utils._primer_add_seqs(fwd_primer)
        kwargs['fwd_primer'] = fwd_primer.sequence
        kwargs['fwd_primer_meta'] = fwd_primer.dump()
        row['Forward Primer'] = fwd_primer.name
    if '_lflank' in row:
        kwargs['lflank'] = row['_lflank']
    if '_rflank' in row:
        kwargs['rflank'] = row['_rflank']
    if '_rev_primer' in row:
        rev_primer = row['_rev_primer']
        primer_utils._primer_add_seqs(rev_primer)
        kwargs['rev_primer'] = rev_primer.sequence
        kwargs['rev_primer_meta'] = rev_primer.dump()
        kwargs['Reverse Primer'] = rev_primer.name
    pair = primer_designer.design_cloning_primers(**kwargs)[0]
    row['Length'] = pair['PAIR']['PRODUCT_SIZE']
    for k1, k2 in (('LEFT', 'Forward Primer'), ('RIGHT', 'Reverse Primer')):
        primer_data = pair[k1]
        if not primer_data['META'].get('id', None):
            primer_row = {
                'Primer': row[k2],
                'Overhang Sequence': primer_data['OVERHANG'],
                'Anneal Sequence': primer_data['SEQUENCE'],
                'T Anneal': round(primer_data['TM'], 2)
            }
            primer_rows.append(primer_row)

In [45]:
def create_primer_df(rows, project):
    primer_columns = ['Primer', 'Project', 'Description'] + [ft.name for ft in aq.SampleType.find_by_name('Primer').field_types]
    df = rows_to_df(rows)
    df['Description'] = ''
    df['Project'] = project
    df = df[primer_columns]
    return df

create_primer_df(primer_rows, 'SD2 2021 Designs')

,Primer,Project,Description,Overhang Sequence,Anneal Sequence,T Anneal
0,3'_TRP_hom__rev,SD2 2021 Designs,,,atggtgtttatgcaaagaaa,51.46
1,3'_URA_hom__rev,SD2 2021 Designs,,,cggcccaagccttgtcccaa,65.49
2,3'_HIS_hom__rev,SD2 2021 Designs,,,tggagccataatgacagcag,57.66
3,TS-3'_HO_hom__fwd,SD2 2021 Designs,,atgtcgtaataaccccgccccg,acgccattttaagtccaaagg,57.07
4,3'_HO_hom__rev,SD2 2021 Designs,,,aattatcctgggcacgagtg,57.67
5,5'_HO_hom__fwd,SD2 2021 Designs,,,TTAGCAGATGCGCGCACCT,62.66
6,5'_HO_hom-pp2__rev,SD2 2021 Designs,,gccgcttttcgaatctggttaagg,GACACTGGATGGCGGCGTTA,62.85
7,TS-3'_ARS314_hom__fwd,SD2 2021 Designs,,atgtcgtaataaccccgccccg,CCTGAGGGGCGCGCCAGATC,67.80
8,3'_ARS314_hom__rev,SD2 2021 Designs,,,TCGAAGAAAATAAACCGCCC,56.43
9,5'_ARS314_hom__fwd,SD2 2021 Designs,,,TGAATGTGTCTCTGATGTACCAG,57.92


In [47]:
def create_fragment_df(rows, project):
    primer_columns = ['Fragment', 'Project', 'Description'] + [ft.name for ft in aq.SampleType.find_by_name('Fragment').field_types]
    df = rows_to_df(rows)
    df['Description'] = ''
    df['Project'] = project
    for c in primer_columns:
        if c not in df.columns:
            df[c] = ''
    df = df[primer_columns]
    return df

create_fragment_df(rows, 'SD2 2021 Designs')

,Fragment,Project,Description,Sequence,Length,Template,Forward Primer,Reverse Primer,Restriction Enzyme(s),Yeast Marker,Fragment Mix Array
0,tCyc1(partial)-TS-TRP_3'hom__v3,SD2 2021 Designs,,,497,pMOD TRP pGalZ4::NLS-eYFP-JAZ12,j5_0436_(pMODkan-HO)_forward,3'_TRP_hom__rev,,,
1,tCyc1(partial)-TS-URA_3'hom__v3,SD2 2021 Designs,,,542,pMOD6-pGRR-RGR-W8,j5_0436_(pMODkan-HO)_forward,3'_URA_hom__rev,,,
2,tCyc1(partial)-TS-HIS_3'hom__v3,SD2 2021 Designs,,,1043,pMOD8A-URGR-W8,j5_0436_(pMODkan-HO)_forward,3'_HIS_hom__rev,,,
3,TS-HO_3'hom__v3,SD2 2021 Designs,,,906,pMOD-HO-KanMX-pRPL18B-rtTA-VPR,TS-3'_HO_hom__fwd,3'_HO_hom__rev,,,
4,HO-KAN-pp2_5'hom__v3,SD2 2021 Designs,,,2038,pMOD-HO-KanMX-pRPL18B-rtTA-VPR,5'_HO_hom__fwd,5'_HO_hom-pp2__rev,,,
5,TS-ARS314-BLEO_3'hom__v3,SD2 2021 Designs,,,2033,pyMOD-BLEO-ARS314.A.0-pGPD-yeVenus-tCYC1,TS-3'_ARS314_hom__fwd,3'_ARS314_hom__rev,,,
6,ARS314-pp2_5'hom__v3,SD2 2021 Designs,,,989,pyMOD-BLEO-ARS314.A.0-pGPD-yeVenus-tCYC1,5'_ARS314_hom__fwd,5'_ARS314_hom-pp2__rev,,,
7,LEU-pp2_5'hom__v3,SD2 2021 Designs,,,2229,pMODL-pGPD-AFB2,5'_LEU_hom__fwd,5'_LEU_hom-pp2__rev,,,
